In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

property_data = {}
obs_data = []

# This dict will map numerical values to actual locations, in order to facilitate
# the creation of the regression model
location_mapping = {}

# Used to fill location_mapping, unique for every different location
location_id = 1

# Update the location mapping with the given string location, and return mapped numerical value
def update_location_mapping(location):
    global location_id
    if(location in location_mapping):
        return location_mapping[location]
    location_mapping[location] = location_id
    location_id += 1
    return location_mapping[location]

# Generate test points for an integer number of years
def generate_test_points(years):
    test_points = []
    current_year = 2017
    end_year = current_year + years
    for year in range(current_year, end_year):
        for loc_string, loc_id in location_mapping.iteritems():
            test_points.append([year, loc_id])
    return test_points

# Given a dict data with field 'input'=[[year, location],...] and 'output' = [obs_value,...]
# return an average obs_value per year as a dict with key year and value obs_value
def average_over_locations(data):
    sums = {}
    counts = {}
    averages = {}
    for i in range(0, len(data['input'])):
        data_point = data['input'][i]
        year = str(data_point[0])       # convert year to string to use as key
        if(year in sums):
            sums[year] += data['output'][i]
            counts[year] += 1
        else:
            sums[year] = data['output'][i]
            counts[year] = 1
    for year in sums:
        averages[year] = sums[year]/counts[year]
    return averages

# Given a dict with key year and value average obs_value, produce a bar graph
def plot_annual_averages(averages, ylabel, title):
    #averages.sort(key=lambda x: x.count, reverse=True)
    fig, ax = plt.subplots()
    years = []
    avg_vals = []
    for year, avg_obs in averages.iteritems():
        years.append(year)
        avg_vals.append(avg_obs)
    years = tuple(years)
    y_pos = np.arange(len(years))
    ax.bar(y_pos, avg_vals)
    plt.bar(y_pos, avg_vals)
    plt.xticks(y_pos, years)
    plt.ylabel(ylabel)
    plt.title(title)
    ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.tick_params(axis='x', which='major', pad=15)
    plt.show()

# Given two dicts, merge them into a new dict as a shallow copy.
def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z

# Initialize connection and query
cnxn = pyodbc.connect('DSN=initial_attempt;uid=ecampbell;pwd=Javascript@110')
cursor = cnxn.cursor()
query = 'SELECT year, location, indicator, obs_value FROM international_data'

# Iterate over row data
for row in cursor.execute(query):
    year = int(row.year)
    obs_value = round(float(row.obs_value))
    location = update_location_mapping(row.location)

    # Update the property data for each type of indicator
    if(row.indicator in property_data):
        property_data[row.indicator]['input'].append([year, location])
        property_data[row.indicator]['output'].append(obs_value)
    else:
        property_data[row.indicator] = {'input': [[year, location]], 'output': [obs_value]}

# Although there's several indicators, we're going to stick with just graphing one
indicator = 'Intellectual Property Rights, Overall (Global Rank)'
input_data = property_data[indicator]['input']
target_data = property_data[indicator]['output']

# Prepare test points in order to produce a predictive model
test_data = generate_test_points(5)

classif = OneVsRestClassifier(estimator=SVC(random_state=0))
predictive_output = classif.fit(input_data, target_data).predict(test_data)

# Calculate averages across all years
averages = merge_two_dicts(average_over_locations(property_data[indicator]),
        average_over_locations({
            'input': test_data,
            'output': predictive_output
        }))
plot_annual_averages(averages, 'OBS Value', 'Intellectual Property Rights Index by Nation')
